# fact checking

In [ ]:
import json
import re

#  saved results
with open('/content/drive/MyDrive/llm/rag_benchmark_results_115.json', 'r') as f:
    results = json.load(f)

In [ ]:


def fact_check_hallucination(generated_answer, expected_answer):
    """
    Focused fact-checking approach to detect medical hallucinations
    """
    # Extract specific medical facts from both answers
    gen_facts = extract_medical_facts(generated_answer)
    ref_facts = extract_medical_facts(expected_answer)

    # Check for contradictions and unsupported claims
    contradictions = []
    unsupported = []

    for fact in gen_facts:
        # Check if this fact contradicts any reference facts
        is_contradicted = False
        has_support = False

        for ref_fact in ref_facts:
            if contradicts(fact, ref_fact):
                contradictions.append(f"Generated: '{fact}' contradicts Reference: '{ref_fact}'")
                is_contradicted = True
                break

            if supports(fact, ref_fact):
                has_support = True

        # If not contradicted but also not supported
        if not is_contradicted and not has_support:
            unsupported.append(f"Unsupported claim: '{fact}'")

    # Calculate hallucination score
    hallucination_score = 0
    if len(gen_facts) > 0:
        # Contradictions are weighted more heavily than unsupported claims
        hallucination_score = (len(contradictions) * 2 + len(unsupported)) / (len(gen_facts) * 2)

    # Determine hallucination level
    if hallucination_score > 0.5 or len(contradictions) > 0:
        level = "High"
    elif hallucination_score > 0.2:
        level = "Medium"
    else:
        level = "Low"

    return {
        "level": level,
        "score": hallucination_score,
        "contradictions": contradictions[:3],  # Limit to top 3
        "unsupported": unsupported[:3]  # Limit to top 3
    }



In [ ]:
def extract_medical_facts(text):
    """Extract medical facts from text"""
    facts = []

    # Extract sentences that likely contain medical facts
    sentences = [s.strip() for s in re.split(r'[.!?]', text) if len(s.strip()) > 10]

    for sentence in sentences:
        # Check for statements that look like medical facts
        if contains_medical_terms(sentence) and is_factual_statement(sentence):
            facts.append(sentence)

    return facts

In [ ]:


def contains_medical_terms(text):
    """Check if text contains medical terminology"""
    medical_terms = [
        "treatment", "symptom", "disease", "condition", "patient",
        "diagnosis", "therapy", "medication", "drug", "dose", "mg", "ml",
        "infection", "bacteria", "virus", "chronic", "acute", "syndrome",
        "prescription", "contraindication", "administered", "clinical"
    ]

    text_lower = text.lower()
    return any(term in text_lower for term in medical_terms)

In [ ]:


def is_factual_statement(text):
    """Check if text appears to be a factual statement"""
    # Factual statements often contain certain patterns
    factual_patterns = [
        r"is a", r"are a", r"causes", r"caused by", r"results in",
        r"leads to", r"is used", r"are used", r"should be", r"can be",
        r"includes", r"consists of", r"recommended", r"prescribed",
        r"indicated for", r"contraindicated"
    ]

    text_lower = text.lower()
    return any(re.search(pattern, text_lower) for pattern in factual_patterns)

In [ ]:


def contradicts(fact1, fact2):
    """Check if two facts contradict each other"""
    # Simple contradiction check - look for opposite statements
    # This is a simplified approach - in reality, contradiction detection is complex

    # Look for negation differences
    if ("not" in fact1 and "not" not in fact2) or ("not" in fact2 and "not" not in fact1):
        # Check if they're talking about the same thing
        fact1_words = set(fact1.lower().split())
        fact2_words = set(fact2.lower().split())
        common_words = fact1_words.intersection(fact2_words)

        # If they share significant words and one has a negation, might be a contradiction
        if len(common_words) / max(len(fact1_words), len(fact2_words)) > 0.5:
            return True

    # Check for different values for the same attribute
    # Example: "The dose is 10mg" vs "The dose is 20mg"
    for pattern in [r"(\d+)\s*mg", r"(\d+)\s*ml", r"(\d+)\s*%"]:
        match1 = re.search(pattern, fact1)
        match2 = re.search(pattern, fact2)

        if match1 and match2 and match1.group(1) != match2.group(1):
            # Check if they're talking about the same thing
            fact1_words = set(fact1.lower().split())
            fact2_words = set(fact2.lower().split())

            if len(fact1_words.intersection(fact2_words)) / max(len(fact1_words), len(fact2_words)) > 0.4:
                return True

    return False

In [ ]:


def supports(fact1, fact2):
    """Check if fact2 supports fact1"""
    # Simple support check - look for similar statements

    # Check word overlap
    fact1_words = set(fact1.lower().split())
    fact2_words = set(fact2.lower().split())

    # If they share significant words, fact2 might support fact1
    return len(fact1_words.intersection(fact2_words)) / max(len(fact1_words), len(fact2_words)) > 0.5

# Process each result and add hallucination assessment
updated_results = []
for item in results:
    hallucination_info = fact_check_hallucination(
        item['generated_answer'],
        item['expected_answer']
    )

    # Add assessment to the result
    updated_item = item.copy()
    updated_item['hallucination_level'] = hallucination_info['level']
    updated_item['hallucination_score'] = hallucination_info['score']
    updated_item['contradictions'] = hallucination_info['contradictions']
    updated_item['unsupported'] = hallucination_info['unsupported']
    updated_results.append(updated_item)

# Save updated results
with open('/content/drive/MyDrive/llm/rag_benchmark_results_fact_checked.json', 'w') as f:
    json.dump(updated_results, f, indent=2)

print(f"Added fact-checking hallucination detection to {len(results)} benchmark results")

# Print summary statistics
hallucination_counts = {"High": 0, "Medium": 0, "Low": 0}
for item in updated_results:
    hallucination_counts[item['hallucination_level']] += 1

print("\nHallucination Summary:")
for level, count in hallucination_counts.items():
    percentage = count / len(updated_results) * 100
    print(f"{level}: {count} ({percentage:.1f}%)")

Added fact-checking hallucination detection to 115 benchmark results

Hallucination Summary:
High: 0 (0.0%)
Medium: 51 (44.3%)
Low: 64 (55.7%)


If a generated answer says:

"The recommended dose is 20mg."
But the reference says:
"The recommended dose is 10mg."

The script will flag this as a contradiction (High hallucination risk).